In [1]:

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from sparkmeasure import StageMetrics
spark=SparkSession \
        .builder \
        .master("spark://ubuntu:7077")\
        .appName("Number of Movies per category which are funny and over 3.5 rating")\
        .getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)

In [2]:
stagemetrics = StageMetrics(spark)

In [3]:
stagemetrics.begin()
Movies_Dataframe=(spark.read.format("csv")
            .option("header","True")
            .option("inferSchema","True")
            .load("/home/meli/Documents/Kaggle/movie.csv")
      )
Ratings_Dataframe=(spark.read.format("csv")
            .option("header","True")
            .option("inferSchema","True")
            .load("/home/meli/Documents/Kaggle/rating.csv")
      )
Tags_Dataframe=(spark.read.format("csv")
            .option("header","True")
            .option("inferSchema","True")
            .load("/home/meli/Documents/Kaggle/tag.csv")
      )
#Format Tags(lowercase and remove specialchars)
Tags_Dataframe=Tags_Dataframe.withColumn('tag',lower(col('tag'))).withColumn('tag',regexp_replace('tag', '[^\w\s]+', ''))
#Format the Movie Dataframe
Mdf=(Movies_Dataframe.select("*", split(col("genres"), "\|"))
                     .drop('genres')
                     .withColumnRenamed('split(genres, \|, -1)','genresar')
    )
Moviedf=Mdf.select("*",explode(Mdf.genresar).alias("Genre")).drop("genresar").sort("Genre",ascending=False)

Ratings_Dataframe=Ratings_Dataframe.filter("rating>3.5").drop('timestamp')
Tags_Dataframe=Tags_Dataframe.filter("tag='funny'").drop('timestamp')
rtdf=Ratings_Dataframe.join(Tags_Dataframe,['movieId','userId'])
rtmdf=rtdf.join(Moviedf,['movieId'])
FullQuery=rtmdf.groupBy("Genre").agg(count('movieId').alias("NumberOfMovies"))
FullQuery.sort('Genre').show()
stagemetrics.end()

+-----------+--------------+
|      Genre|NumberOfMovies|
+-----------+--------------+
|     Action|           366|
|  Adventure|           402|
|  Animation|           255|
|   Children|           257|
|     Comedy|          1336|
|      Crime|           231|
|Documentary|            20|
|      Drama|           487|
|    Fantasy|           268|
|  Film-Noir|             2|
|     Horror|           112|
|       IMAX|            69|
|    Musical|            88|
|    Mystery|            81|
|    Romance|           412|
|     Sci-Fi|           175|
|   Thriller|           205|
|        War|            23|
|    Western|            41|
+-----------+--------------+



In [4]:
stagemetrics.print_report()


Scheduling mode = FIFO
Spark Context default degree of parallelism = 4
Aggregated Spark stage metrics:
numStages => 10
numTasks => 225
elapsedTime => 57925 (58 s)
stageDuration => 57811 (58 s)
executorRunTime => 152563 (2.5 min)
executorCpuTime => 66347 (1.1 min)
executorDeserializeTime => 13049 (13 s)
executorDeserializeCpuTime => 3255 (3 s)
resultSerializationTime => 494 (0.5 s)
jvmGCTime => 1996 (2 s)
shuffleFetchWaitTime => 0 (0 ms)
shuffleWriteTime => 320 (0.3 s)
resultSize => 1511723 (1476.0 KB)
diskBytesSpilled => 0 (0 Bytes)
memoryBytesSpilled => 0 (0 Bytes)
peakExecutionMemory => 211812352
recordsRead => 30981363
bytesRead => 1428438922 (1362.0 MB)
recordsWritten => 0
bytesWritten => 0 (0 Bytes)
shuffleRecordsRead => 106
shuffleTotalBlocksFetched => 106
shuffleLocalBlocksFetched => 106
shuffleRemoteBlocksFetched => 0
shuffleTotalBytesRead => 7668 (7.0 KB)
shuffleLocalBytesRead => 7668 (7.0 KB)
shuffleRemoteBytesRead => 0 (0 Bytes)
shuffleRemoteBytesReadToDisk => 0 (0 Bytes)
s